In [3]:
import numpy as np
import pandas as pd

In [4]:
# CIR model
def CIR(dt,T,r0,sigma,k,mean_r):
    r = []
    r.append(r0)
    n = int(round(T/dt))
    w = np.random.normal(0,1,n)
    for i in range(0,n):
        if r[i] >= 0 :
            ri = r[i] + k*(mean_r-r[i])*dt + sigma*np.sqrt(dt*r[i])*w[i]
        else:
            ri = r[i] + k*(mean_r-r[i])*dt # in case negative r pump out
        r.append(ri)
    return r

In [5]:
# Numerix Prepayment Model
def NumerixPrepayment(R,pre_r10,pre_PV,PV0,t,month):
    month = int(month)
    RIt = 0.28 + 0.14*np.arctan(-8.57+430*(R-pre_r10))
    BUt = 0.3 + 0.7*(pre_PV/PV0)
    SGt = min(1,t/30)
    SYt = [0.94,0.76,0.74,0.95,0.98,0.92,0.98,1.10,1.18,1.22,1.23,0.98][month-1]
    CPRt = RIt * BUt * SGt * SYt
    return CPRt

In [6]:
def MBS_Price(dt,T,r0,sigma,k,mean_r,R,PV0,M,OAS):
    
    p = []
    IO = []
    PO = []
    
    for m in tqdm(range(0,M)):
        
        rf = CIR(dt,T,r0,sigma,k,mean_r) # simulate rf 
        Rf = (pd.Series(rf)+OAS).cumsum()*dt # discount rate
        n_m = int(round((1/12)/dt)) # how many dt in one month
        r_m = R/12 # mortgage rate per month
        N = T*12
        PV = []
        PV.append(PV0)
        Ct = []
        disc_rate = []
        IOt = []
        POt = []
        
        # Parameters of CIR model
        h1 = np.sqrt(k**2+2*sigma**2)
        h2 = (k+h1)/2
        h3 = 2*k*mean_r/(sigma**2)
        At = ((h1*np.exp(h2*10))/(h2*(np.exp(h1*10)-1)+h1)) ** h3
        Bt = (np.exp(h1*10)-1)/(h2*(np.exp(h1*10)-1)+h1)
        
        for t in range(1,N+1):
            
            # find the month at t
            if t % 12 == 0:
                month = 12
            else:
                month = t % 12
            
            t_yr = t/12 # t in yr unit
            P_bond = At * np.exp(-Bt*rf[n_m*(t-1)]) # price of 10yr bond
            pre_r10 = (-1/10) * np.log(P_bond) # 10 yr risk-free rate
            CPRt = NumerixPrepayment(R,pre_r10,PV[t-1],PV0,t_yr,month) 
            
            PMTt = (PV[t-1] * r_m) / (1-1/((1+r_m)**(N-(t-1)))) # PMT
            IPt = PV[t-1] * r_m # interest payment
            SPt = PMTt - IPt # scheduled pricipal payment
            PPt = (PV[t-1]-SPt) * (1-(1-CPRt)**(1/12)) # prepayment
            
            PV.append(PV[t-1]-SPt-PPt) # PVt
            Ct.append(SPt+PPt+IPt) # cash flow
            IOt.append(IPt) # Interest-only
            POt.append(SPt+PPt) # Principal-only
            disc_rate.append(Rf[n_m*t]) # discount rate
        
        pi = sum(pd.Series(Ct)*np.exp(-1*pd.Series(disc_rate)))
        IOi = sum(pd.Series(IOt)*np.exp(-1*pd.Series(disc_rate)))
        POi = sum(pd.Series(POt)*np.exp(-1*pd.Series(disc_rate)))
        p.append(pi)
        IO.append(IOi)
        PO.append(POi)
        
    return np.mean(p),np.mean(IO),np.mean(PO)